In [25]:
from tqdm import tqdm # progress bar when long task
import pandas as pd
import numpy as np
import pickle
import re
import os

from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords
from nltk import Counter
import spacy
nlp = spacy.load("en")

## LyricsPreprocessing CLASS

In [19]:
class LyricsPreprocessing():
    
    
    def __init__(self, lyrics_df):
        
        self.lyrics_df = lyrics_df
        
        
    def preliminary_preprocessing(self, lyrics_df=None, replace_number_with="0", every_digit=True):
        
        if lyrics_df is None:
            lyrics_df = self.lyrics_df
        
        # lowercase
        print("Converting to lower case ...", flush=True)
        lyrics_df.loc[:, "lyrics"] = lyrics_df.lyrics.apply(lambda text: text.strip().lower())
        
        # replace numbers
        print(f"Replacing numbers with {replace_number_with} ...", flush=True)
        if every_digit:
            numbers = re.compile('[0-9]+')
        else:
            numbers = re.compile('[0-9]')
        
        lyrics_df.loc[:, "lyrics"] = lyrics_df.lyrics.apply(lambda text: re.sub(numbers, replace_number_with, text))
        
        return lyrics_df

    
    def tokenize(self, lyrics_df=None, save=None):
        
        if lyrics_df is None:
            lyrics_df = self.lyrics_df
        
        print("Tokenizing ...", flush=True)
        tokenized_corpus = [[token.text for token in nlp(lyrics)] for lyrics in tqdm(lyrics_df.loc[:, "lyrics"].to_list())]
        self.tokenized_corpus = tokenized_corpus
        
        if not save is None:
            self.save(tokenized_corpus, 'tokenized_corpus', how=save)
        
        return tokenized_corpus
    
    
    def core_preprocessing(self, lyrics_df, corpus):
        
        if lyrics_df is None:
            lyrics_df = self.lyrics_df
            
        trimmed_corpus = []
        tagged_corpus = []
        for lyrics in tqdm(lyrics_df.loc[:, "lyrics"].to_list()):
            trimmed_lemmas = []
            tagged_lemmas = []
            for token in nlp(lyrics):
                # remove all tokens with length <= 2
                if len(str(token)) > 2:
                    # lemmatize and get POS
                    lemma = token.lemma_
                    pos = token.pos_
                    # keep only if POS in ['NOUN', 'VERB', 'ADJ', 'ADV','INTJ', 'X']
                    if pos in ['NOUN', 'VERB', 'ADJ', 'ADV', 'INTJ', 'X']:
                        trimmed_lemmas.append(lemma)
                        tagged_lemmas.append(lemma + "_" + pos)
            trimmed_corpus.append(trimmed_lemmas)
            tagged_corpus.append(tagged_lemmas)
        
        self.trimmed_corpus = trimmed_corpus
        self.tagged_corpus = tagged_corpus
        
           
    def join_collocations(self, preprocessed_corpus, most_common=20):
        
        # separate each lyrics with a space in order to avoid collocations between different songs
        words = []
        for idx, lyrics in enumerate(preprocessed_corpus):
            # not for the first
            if idx > 0:
                words.append(' ')
            for token in lyrics:
                words.append(token)
            
        finder = BigramCollocationFinder.from_words(words)
        bgm = BigramAssocMeasures()
        score = bgm.mi_like
        collocations = {'_'.join(bigram): pmi for bigram, pmi in finder.score_ngrams(score)}
        
        return Counter(collocations).most_common(most_common)
    
    
    def save(self, obj, filename, path='/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/data/', how='pickle'):
        
        if how == 'pickle':
            with open(os.path.join(path, filename+'.pickle'), 'rb') as pickled_obj:
                pickle.dump(obj, pickled_object)
               

In [20]:
lyrics_preprocess = LyricsPreprocessing(lyrics_df=lyrics_df)

In [23]:
lyrics_cleaned = lyrics_preprocess.preliminary_preprocessing()
lyrics_cleaned.iloc[4758, :]

Converting to lower case ...
Replacing numbers with  ...


artist                                          Bruce Springsteen
song                                         Racing In The Street
album                                Darkness on the Edge of Town
release_date                                           1978-06-02
genre                                                        Rock
lyrics          i got a sixty-nine chevy with a 0 fuelie heads...
year                                                         1978
Name: 4758, dtype: object

In [27]:
# tokenization took some time to finish. Let's pickle it to make it easily reusable
with open('./data/trimmed_corpus.pickle', 'rb') as pickled_object:
    trimmed_corpus = pickle.load(pickled_object)

In [33]:
genre_idxs = lyrics_df.loc[lyrics_df.genre == "Hip-Hop", :].index

In [34]:
lyrics_preprocess.join_collocations(np.array(trimmed_corpus)[genre_idxs].tolist(), most_common=100)

[('la_la', 428.0013218590605),
 ('ooh_ooh', 157.50899525998275),
 ('hip_hop', 59.791212630751424),
 ('zoote_zoote', 54.90012558662172),
 ('da_da', 49.29725050967009),
 ('no_matter', 38.71592661934151),
 ('o_o', 38.685505609873736),
 ('boom_boom', 37.862219323102245),
 ('unfazed_unfazed', 37.5741965973535),
 ('oo_oo', 37.03362978638222),
 ('tick_tock', 32.255857966834895),
 ('at_all', 28.039869258449368),
 ('ling_ling', 27.428571428571427),
 ('icky_icky', 27.34375),
 ('bye_bye', 26.885108170552407),
 ('brand_new', 24.787714543812104),
 ('sensual_seduction', 23.335714285714285),
 ('diddly_diddly', 23.03810650887574),
 ('motivated_motivated', 22.345270890725438),
 ('doo_doo', 22.07128460091373),
 ('too_much', 21.809510647908752),
 ('thicky_thicky', 20.444444444444443),
 ('•_•', 20.301854191438764),
 ('doot_doot', 20.195763330898465),
 ('win_win', 19.419370153701983),
 ('minor_setback', 18.889068825910933),
 ('lovey_dovey', 18.776234567901234),
 ('knooow_beee', 18.0),
 ('heya_heya', 17.998

# STATISTICS 

## Tokens 

## Types

## Collocations